# boto3 library


`boto3` is the Python SDK for AWS (Amazon Web Service).  
It is the python library used to interact with all AWS services (S3, EC2, DynamoDB...). 

This SDK contains an extremely large number of methods, all with a lot of options, across all available AWS services.   
This tutorial just shows the main concepts and code samples for a few actions.  
For a full guide on this library, refer to the official AWS documentation : https://boto3.amazonaws.com/v1/documentation/api/latest/index.html


### Setup

To use the AWS SDK, we need to be authenticated in an AWS account.  
- create an AWS account if needed
- install the AWS CLI locally
- run `aws configure` from a local terminal to save the AWS account access/secret keys under `~/.aws/credentials` and the default region under `~/.aws/config`

Alternatively, you can just open an AWS CloudShell from the AWS console and use `boto3` from a python script inside it.  
The CloudAWS Shell comes with a pre-installed AWS CLI and inherits the credentials from the logged user by default.


### Concepts of the boto3 SDK

A **client** is a low-level interface to an AWS service offering methods for each possible API call.  
This is the original way to call the AWS API from Python.

A **resource** is an object-oriented interface to an AWS service providing a higher level of abstraction.  
The resource has a number of attributes and methods specific to the service they apply to.  
The resource API is newer and does not support yet all possible API calls.


### Example with an S3 client

The below code creates a new bucket in S3, adds a file in it, then remove the file and delete the bucket using the `client` interface.

In [201]:
import boto3
import requests
import time


# Create an S3 client
s3 = boto3.client('s3')

# Iterable list of buckets in this account
s3.list_buckets()['Buckets']

# Create a new bucket
bucket_name = 'test-bucket-' + str(int(time.time()))   # globally unique bucket name
location    = {'LocationConstraint': 'ap-northeast-1'}
s3.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration=location)

# Create a file in this bucket
file_name = 'test.txt'
s3.put_object(
    Body=file_name,
    Bucket=bucket_name,
    Key=file_name)

# Delete the file in this bucket
s3.delete_object(Bucket=bucket_name, Key=file_name)

# Delete the bucket
s3.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'S60SPQ0HQ3QK8CGK',
  'HostId': 'fpmrR5EkKo91Tn7+UGdQTmln3YLxDYBY8qejjSxgPFkIOLBmRKw09j1suVq54KvVu/WR6VOhaWg=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'fpmrR5EkKo91Tn7+UGdQTmln3YLxDYBY8qejjSxgPFkIOLBmRKw09j1suVq54KvVu/WR6VOhaWg=',
   'x-amz-request-id': 'S60SPQ0HQ3QK8CGK',
   'date': 'Sun, 04 Jul 2021 07:12:42 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Example with an S3 resource

The below code performs the exact same actions as above, but using a `resource` instead of a `client` interface.  
Instead of performing all API calls by providing a resource ID, we now have a `Bucket` and an `Object` class to represent the S3 concepts.

In [202]:
# Create an S3 resource
s3 = boto3.resource('s3')

# Iterable list of buckets
# With the resource interface we have a "buckets" attribute wrapping the API call
s3.buckets.all()

# Create a new bucket
# With the resource interface, we can instanciate a new Bucket object
bucket_name = 'test-bucket-' + str(int(time.time()))   # globally unique bucket name
location    = {'LocationConstraint': 'ap-northeast-1'}
s3.Bucket(name=bucket_name).create(CreateBucketConfiguration=location)

# Create a file in this bucket    
file_name = 'test.txt'
s3.Object(bucket_name, file_name).put(Body=open(file_name, 'rb'))

# Delete the file in this bucket
s3.Bucket(name=bucket_name).delete_objects(
    Delete={
        'Objects': [
            {'Key': file_name}
        ]
    })

# Delete this bucket
s3.Bucket(name=bucket_name).delete()

{'ResponseMetadata': {'RequestId': 'PG6WWW6EBN01WB15',
  'HostId': 'LPtmW4vQaKJGeCo/fkoOR0A0wGetMQwGZqK3K/jcnJzYAAk7DzZEQzw69YdifH1/5sMlLQhynH0=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'LPtmW4vQaKJGeCo/fkoOR0A0wGetMQwGZqK3K/jcnJzYAAk7DzZEQzw69YdifH1/5sMlLQhynH0=',
   'x-amz-request-id': 'PG6WWW6EBN01WB15',
   'date': 'Sun, 04 Jul 2021 07:12:44 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Sample SQS code

The below code interacts with the SQS service (Simple Queue service) and performs the following actions:
- Create an SQS queue
- Ensure the queue is empty
- Send a message to the queue
- Poll the message from the queue
- Deletes the message from the queue
- Deletes the queue

In [203]:
# Create the client
sqs = boto3.client('sqs')

# Create a queue
queue_name = 'boto3-test-queue-' + str(int(time.time()))
queue      = sqs.create_queue(QueueName=queue_name)
queue_url  = queue['QueueUrl']

# Poll a message from the queue
response = sqs.receive_message(QueueUrl=queue_url)
assert 'Messages' not in response     # No actual message returned since the queue is empty

# Send a message to the queue
sqs.send_message(
    QueueUrl=queue_url,
    MessageAttributes={
        'Title': {
            'DataType': 'String',
            'StringValue': 'The Three-Body Problem'
        },
        'Author': {
            'DataType': 'String',
            'StringValue': 'Liu Cixin'
        }
    },
    MessageBody='Book Suggestion'
)

# Receive a message
response = sqs.receive_message(
    QueueUrl=queue_url,
    MaxNumberOfMessages=1,
    MessageAttributeNames=['All'],   # By default, do not fetch the message attribute
    VisibilityTimeout=10)

assert 'Messages' in response
msg = response['Messages'][0]
assert msg['Body'] == 'Book Suggestion'
assert msg['MessageAttributes']['Author']['StringValue'] == 'Liu Cixin'

# Delete the message from the queue (so it cannot be polled again)
receipt = msg['ReceiptHandle']
print(receipt)
sqs.delete_message(
    QueueUrl=queue_url,
    ReceiptHandle=receipt
)

# Delete the queue
sqs.delete_queue(QueueUrl=queue['QueueUrl'])

AQEBQM7pC1Sq4HLwhNxembkbMJHN5P4JFNZPpawCNZ83npRmJ8QoV5KQok67qeP7KRAklghDinV719iXCnJVR01SWqJq6BLqXKXtInVy5bgsVWUeJH9hGVy0Lf96815IkG8zt6OL3YjH2FpQ0hw8iz9cs9Ih+tydCJvmcD7YPN5G/r6xKphTxmFxLZcODABilgEC/8n/dbdiuYlNl3+KWgnVLuHibnuhXBSzQcBbiohfZ42Bay/QLrt0c0f69QWiMp/xsLYqGJLWRObRq/OlF2g7VtmhtkJ9fxcUJEkjANZprlhxMjdPqL+3XJXS9606dmt5XN67hkyBSwhoka+A1KvR26SCk9mzeCijDJs5niJkui/WK8tIIZy+N6UajZJUhXhGxNAHosYntqx3ODTnQaL/ijnNzwUjz85XYxkcmZAyDj4=


{'ResponseMetadata': {'RequestId': 'bde1c41b-9adc-5af1-aa97-9d40fb703dee',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bde1c41b-9adc-5af1-aa97-9d40fb703dee',
   'date': 'Sun, 04 Jul 2021 07:12:44 GMT',
   'content-type': 'text/xml',
   'content-length': '211'},
  'RetryAttempts': 0}}

### Sample EC2 code

The below code interacts with the EC2 service and performs the following actions :
- Create a security group in the default VPC
- Add some rules in the security group to allow HTTP, HTTPS and SSH ingress traffic
- Create an EC2 instance running a simple Apache web server (in the security group created above)
- Wait for the EC2 instance to be running and the web server to be ready
- Query the web server
- Shutdown the EC2 instance and wait for its termination
- Delete the security group

In [204]:
# Create an EC2 client
ec2 = boto3.resource('ec2')

# Create a security group
sg_name = 'boto3-sg'
sg = ec2.create_security_group(
   Description='Test SG created from boto3',
   GroupName=sg_name
)

# Allow inbound traffic on port 80 (HTTP), 443 (HTTPS) amd 22 (SSH) for the security group
sg.authorize_ingress(
    IpPermissions=[
        {
            'FromPort': 80,
            'ToPort': 80,
            'IpProtocol': 'tcp',
            'IpRanges': [{ 'CidrIp': '0.0.0.0/0', 'Description': 'HTTP public access'}]
        },
        {
            'FromPort': 443,
            'ToPort': 443,
            'IpProtocol': 'tcp',
            'IpRanges': [{ 'CidrIp': '0.0.0.0/0', 'Description': 'HTTPS public access'}]
        },
        {
            'FromPort': 22,
            'ToPort': 22,
            'IpProtocol': 'tcp',
            'IpRanges': [{ 'CidrIp': '0.0.0.0/0', 'Description': 'SSH public access'}]
        }
    ])

# Start an EC2 instance 
image_id = 'ami-06631ebafb3ae5d34'   # latest Amazon Linux 2 AMI
user_data = """#!/bin/bash
yum update -y
yum install -y httpd.x86_64
systemctl start httpd.service
systemctl enable httpd.service
echo "Hello World from $(hostname -f)" > /var/www/html/index.html"""

instances = ec2.create_instances(
    ImageId=image_id,
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    SecurityGroups=[sg_name],
    KeyName='boto3-key',
    UserData=user_data
)
instance = instances[0]

# Wait for the EC2 instance to be running
print('Waiting for the EC2 instance to be running...')
start = time.time()
instance.wait_until_running()
end = time.time()
print('The EC2 instance is running after {0} seconds.'.format(int(end-start)))

# Get the public IPv4 of the EC2 instance
instance.load()       # load the lazy-loading attributes (including public IPv4)
public_ipv4 = instance.public_ip_address
private_dns = instance.private_dns_name
print('Public IPv4: {0}'.format(public_ipv4))
print('Private DNS: {0}'.format(private_dns))

# Query the web server on the EC2 instance
# Once the EC2 instance is up, it takes about a minute for the web server to be running
url = 'http://{0}'.format(public_ipv4)
res = None
while res is None:
    try:
        res = requests.get(url)
        print('The web server returned : ' + res.text)
    
    except Exception as e:
        print('The web server is not started yet, waiting for 10s...')
        time.sleep(10)
        
# Terminate the EC2 instance
instance.terminate()
instance.wait_until_terminated()

# Delete the security group
sg.delete()

Waiting for the EC2 instance to be running...
The EC2 instance is running after 45 seconds.
Public IPv4: 54.178.27.40
Private DNS: ip-172-31-46-59.ap-northeast-1.compute.internal
The web server is not started yet, waiting for 10s...
The web server is not started yet, waiting for 10s...
The web server is not started yet, waiting for 10s...
The web server is not started yet, waiting for 10s...
The web server is not started yet, waiting for 10s...
The web server returned : Hello World from ip-172-31-46-59.ap-northeast-1.compute.internal



{'ResponseMetadata': {'RequestId': 'ec9c90f0-9461-4017-a767-1e49b789e7c9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ec9c90f0-9461-4017-a767-1e49b789e7c9',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '239',
   'date': 'Sun, 04 Jul 2021 07:15:27 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}